In [5]:
import pymongo
from pymongo import MongoClient,ReturnDocument
from bson import ObjectId
import requests
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path,pdfinfo_from_path
from pdf2image.exceptions import PDFSyntaxError,PDFPageCountError
import os
import glob
import time

In [6]:
def download_pdf(url,storage_dir,title):
    r = requests.get(url, allow_redirects=True)
    pdf_path = f"./{storage_dir}/{title}.pdf"
    open(f"./{storage_dir}/{title}.pdf", 'wb').write(r.content)
    return pdf_path
    
def pdf_to_images(pdf_path,storage_dir,object_id,mongodb_collection):
    info = pdfinfo_from_path(pdf_path)
    maxPages = info["Pages"]
    mongodb_collection.update_one({'_id':object_id},
                                {"$set": {"numb_pages": maxPages}},
                                )    
    image_counter = 0
    for page in range(1, maxPages+1, 10): 
        pages = convert_from_path(pdf_path, dpi=200, first_page=page, last_page = min(page+10-1,maxPages))
        
        for individual_page in pages:
            filename = f"./{storage_dir}/page_{image_counter}.jpg"
            individual_page.save(filename, 'JPEG')
            image_counter += 1

def images_to_text(storage_dir):
    allfiles = os.listdir(storage_dir)
    images = [filename for filename in allfiles if filename.endswith('.jpg')]
    
    master_text = ''
    for img_number in range(0,len(images)): 
        filename = f"./{storage_dir}/page_{img_number}.jpg"
        master_text += pytesseract.image_to_string(Image.open(filename))
        
    return master_text

def upload_text_to_mongodb(mongodb_collection,object_id,master_text):
    mongodb_collection.update_one({'_id':object_id},
                                {'$set': {'full_text': master_text}},
                                )
def delete_temp_content(storage_dir):
    files = glob.glob(f'./{storage_dir}/*')
    for f in files:
        os.remove(f)

In [17]:
client = MongoClient('localhost', 27017)
db = client.db_world_bank
reports_coll = db["reports"]

#.batchSize(100)

#coll_knowledge_notes = reports_coll.find({'collection':'Knowledge Notes'},no_cursor_timeout=True)

start_index = 1262

for iteration in range(20):
    coll_knowledge_notes = reports_coll.find({'collection':'Knowledge Notes'},no_cursor_timeout=True)
    for element in coll_knowledge_notes[start_index:start_index+99]:
        print(start_index)
        url = element["pdf_link"]
        object_id = element['_id']
        storage_dir = 'tempstorage'
        title = element["title"]
        start_index+=1

        print(url)
        print(title)
        if url:
            if 'full_text' not in element.keys():
                try:
                    pdf_path = download_pdf(url,storage_dir,title)
                    pdf_to_images(pdf_path,storage_dir,object_id,reports_coll)
                    master_text = images_to_text(storage_dir)
                    upload_text_to_mongodb(reports_coll,object_id,master_text)
                    delete_temp_content(storage_dir)
                    time.sleep(60)
                except:
                    print('ERROR - trying again')
                    time.sleep(300)
                    try:
                        pdf_path = download_pdf(url,storage_dir,title)
                        pdf_to_images(pdf_path,storage_dir,object_id,reports_coll)
                        master_text = images_to_text(storage_dir)
                        upload_text_to_mongodb(reports_coll,object_id,master_text)
                        delete_temp_content(storage_dir)
                    except:
                        print('ERROR AGAIN WITH THIS DOCUMENT - MOVING ON')
                        pass


https://openknowledge.worldbank.org/bitstream/handle/10986/22417/FONDEN000Mexic0isaster0expenditures.pdf?sequence=1&isAllowed=y
FONDEN--Mexico’s National Disaster Fund : An Evolving Inter-Institutional Fund for Post-Disaster Expenditures
https://openknowledge.worldbank.org/bitstream/handle/10986/16140/801010drm0kn1030Box0377295B00PUBLIC0.pdf?sequence=5&isAllowed=y
Hydro-meteorological Disasters Associated with Tsunamis and Earthquakes
https://openknowledge.worldbank.org/bitstream/handle/10986/17046/NonAsciiFileName0.pdf?sequence=1&isAllowed=y
https://openknowledge.worldbank.org/bitstream/handle/10986/29697/125046-BRI-Fiscal-Transparency-GPSM-4-PUBLIC.pdf?sequence=1&isAllowed=y
The Current State of Fiscal Transparency : Norms, Assessment, and Country Practices
https://openknowledge.worldbank.org/bitstream/handle/10986/29698/125050-BRI-Right-to-Information-GPSM-5-PUBLIC.pdf?sequence=1&isAllowed=y
Implementing Right to Information
https://openknowledge.worldbank.org/bitstream/handle/10986

https://openknowledge.worldbank.org/bitstream/handle/10986/22616/809130BRI0MENA070Box379814B00OUO090.pdf?sequence=1&isAllowed=y
Managing Credit Risk in Microfinance : Challenges in the Wake of the Arab Spring
https://openknowledge.worldbank.org/bitstream/handle/10986/18684/740560BRI00Jum0Box0379795B00PUBLIC0.pdf?sequence=1&isAllowed=y
Using Evidence to Scale Up Innovation : Insights from a Results-Based Financing Project for Health in Zambia
https://openknowledge.worldbank.org/bitstream/handle/10986/17065/760640BRI0Nuts00Box374357B00PUBLIC0.pdf?sequence=1&isAllowed=y
Using M&E to Support Performance Based Planning and Budgeting in Indonesia
https://openknowledge.worldbank.org/bitstream/handle/10986/16171/730030BRI0Quic0C0disclosed010010120.pdf?sequence=1&isAllowed=y
Privatization : Lessons from Jordan
https://openknowledge.worldbank.org/bitstream/handle/10986/17034/756560BRI0EP1080Box374342B00PUBLIC0.pdf?sequence=1&isAllowed=y
What Promises Does the Eurasian Customs Union Hold for the 

https://openknowledge.worldbank.org/bitstream/handle/10986/17019/810760BRI0E2P00Box0379828B00PUBLIC0.pdf?sequence=5&isAllowed=y
What's the Long-Term Impact of Conditional Cash Transfers on Education?
https://openknowledge.worldbank.org/bitstream/handle/10986/25487/98814-BRI-VC-ADD-SERIES-PUBLIC-Box393182B.pdf?sequence=1&isAllowed=y
Strengthening Cabinet Office Procedures
https://openknowledge.worldbank.org/bitstream/handle/10986/16114/802500BRI0MENA0Box0379802B00PUBLIC0.pdf?sequence=1&isAllowed=y
MENA and Public Consultations
https://openknowledge.worldbank.org/bitstream/handle/10986/17051/750360BRI0EP1040Box374318B00PUBLIC0.pdf?sequence=1&isAllowed=y
Trade Costs and Development : A New Data Set
https://openknowledge.worldbank.org/bitstream/handle/10986/25458/755620BRI0ARGP00Box374337B00PUBLIC0.pdf?sequence=1&isAllowed=y
Empowering Adolescent Girls in Uganda
https://openknowledge.worldbank.org/bitstream/handle/10986/16156/793700BRI0drm000Box377374B00Public0.pdf?sequence=1&isAllowed=y
G

https://openknowledge.worldbank.org/bitstream/handle/10986/21832/954070WP0Azerb00Box391416B00PUBLIC0.pdf?sequence=1&isAllowed=y
Republic of Azerbaijan : Climate Change and Agriculture Country Note
https://openknowledge.worldbank.org/bitstream/handle/10986/10057/673450Revised00Notes0Teachers0Press.pdf?sequence=1&isAllowed=y
Learning from the Best : Improving Learning Through Effective Teacher Policies
https://openknowledge.worldbank.org/bitstream/handle/10986/17060/NonAsciiFileName0.pdf?sequence=1&isAllowed=y
Improving Governance and Management of Health Systems : Partnerships and Observatories in Latin America and the Caribbean
https://openknowledge.worldbank.org/bitstream/handle/10986/17061/NonAsciiFileName0.pdf?sequence=1&isAllowed=y
Public Spending for Long-Run Growth : A Practitioners' View
https://openknowledge.worldbank.org/bitstream/handle/10986/17059/NonAsciiFileName0.pdf?sequence=1&isAllowed=y
Do Wage Subsidies Help Young Women Get Jobs?
https://openknowledge.worldbank.org/bit

https://openknowledge.worldbank.org/bitstream/handle/10986/17064/752380BRI0SMAR0dge0in0Ghana0nov2012.pdf?sequence=1&isAllowed=y
How to Revamp a Business Edge Program : The Case of Ghana
https://openknowledge.worldbank.org/bitstream/handle/10986/10844/677410BRI00PUB020120Box367893B0QN60.pdf?sequence=1&isAllowed=y
Opening Doors : Gender Equality in the Middle East And North Africa
https://openknowledge.worldbank.org/bitstream/handle/10986/20571/752170BRI0Box30ial0Use0Only0600QN80.pdf?sequence=1&isAllowed=y
The Palestinian Information Technology Association of Companies : Moving Forward on Information and Communications Technology and Innovation
https://openknowledge.worldbank.org/bitstream/handle/10986/17043/762150BRI0Box30OWLEDGE0NOTES0SERIES.pdf?sequence=1&isAllowed=y
Cities as Drivers of Growth along the Silk Road
https://openknowledge.worldbank.org/bitstream/handle/10986/17045/751890BRI0Box30NOWLEDGE0NOTE0SERIES.pdf?sequence=1&isAllowed=y
Governance of Multi-sectoral Interventions to

...

https://openknowledge.worldbank.org/bitstream/handle/10986/10857/661100BRI0Box30Informality0revised1.pdf?sequence=1&isAllowed=y
Striving for Better Jobs : The Challenge of Informality in the Middle East and North Africa Region
https://openknowledge.worldbank.org/bitstream/handle/10986/10423/682030BRI00PUB00Ukraine0Food0Safety.pdf?sequence=1&isAllowed=y
GO LITE! Increasing Scale and Impact by Combining Diagnostics and Training Lessons from the Ukraine Food Safety Project
https://openknowledge.worldbank.org/bitstream/handle/10986/25527/755600BRI0ARGP00Box374337B00PUBLIC0.pdf?sequence=1&isAllowed=y
Environmental and Gender Impacts of Land Tenure Regularization in Africa : Pilot Evidence from Rwanda
https://openknowledge.worldbank.org/bitstream/handle/10986/10848/677400BRI00PUB020120Box367893B0QN59.pdf?sequence=1&isAllowed=y
Tackling the Shelter Challenge : Developing the Mortgage Market in Egypt
https://openknowledge.worldbank.org/bitstream/handle/10986/10058/673460revised000110Assessment

https://openknowledge.worldbank.org/bitstream/handle/10986/11110/516310BRI0Fina1Box342046B001PUBLIC1.pdf?sequence=1&isAllowed=y
The Global Financial Crisis : Comparisons with the Great Depression and Scenarios for Recovery
https://openknowledge.worldbank.org/bitstream/handle/10986/10104/604560BRI0231R10BOX358322B01PUBLIC1.pdf?sequence=1&isAllowed=y
Disaster Risk Financing : Case Studies
https://openknowledge.worldbank.org/bitstream/handle/10986/11683/648550WSP0Tanz00Box361546B00PUBLIC0.pdf?sequence=1&isAllowed=y
Tanzania : A Handwashing Behavior Change Journey
https://openknowledge.worldbank.org/bitstream/handle/10986/11686/634150revised00LIC00SSNPrimerNote32.pdf?sequence=1&isAllowed=y
Natural Disasters : What is the Role for Social Safety Nets?
https://openknowledge.worldbank.org/bitstream/handle/10986/26519/6583400WP0REPL0web0Guidance0Note002.pdf?sequence=1&isAllowed=y
Integrating Gender Issues in Disaster Risk Management Policy Development and in Projects
https://openknowledge.world

https://openknowledge.worldbank.org/bitstream/handle/10986/17110/732010BRI0PPI0010Global0update0note.pdf?sequence=1&isAllowed=y
Private Activity in Infrastructure Slowed Down in the First Half of 2011
https://openknowledge.worldbank.org/bitstream/handle/10986/11051/665550BRI00PUB0e0Online0Discussion1.pdf?sequence=1&isAllowed=y
Summary of the Online Discussion on Linking Gender, Poverty, and Environment for Sustainable Development (May 2 - June 17, 2011)
https://openknowledge.worldbank.org/bitstream/handle/10986/10224/530440BRI0Cris10Box345594B01PUBLIC1.pdf?sequence=1&isAllowed=y
The Leverage Ratio : A New Binding Limit on Banks
https://openknowledge.worldbank.org/bitstream/handle/10986/10428/681850BRI00PUB0riculturetotheClouds.pdf?sequence=1&isAllowed=y
Taking Haitian Agriculture to the Clouds : Implementing Google Apps for Government at the Ministry of Agriculture
https://openknowledge.worldbank.org/bitstream/handle/10986/10071/655360BRI0worldbank0Box361565B00PUBLIC0.pdf?sequence=1&is

https://openknowledge.worldbank.org/bitstream/handle/10986/17229/720970REPLACEM0nitation0East0Africa.pdf?sequence=1&isAllowed=y
Scaling Up Rural Sanitation : Partnering on the Road Towards Achieving Total Sanitation in East Africa
https://openknowledge.worldbank.org/bitstream/handle/10986/10427/677090BRI00PUB0htoMakeMoneyinAfrica.pdf?sequence=1&isAllowed=y
Growing Fish to Make Money in Africa
https://openknowledge.worldbank.org/bitstream/handle/10986/10426/657660BRI0KenyaE2P0Box365727B00PUBLIC0.pdf?sequence=1&isAllowed=y
Do Vouchers for Job Training Programs Help?
https://openknowledge.worldbank.org/bitstream/handle/10986/10434/648790BRI0IFC00iang0Employing0Women.pdf?sequence=1&isAllowed=y
Employing Women : A Boon for a Chemical Plant in India
https://openknowledge.worldbank.org/bitstream/handle/10986/10078/651010BRI00PUB0nBreve01710Printable.pdf?sequence=1&isAllowed=y
Non-Communicable Diseases in Jamaica : Moving from Prescription to Prevention
https://openknowledge.worldbank.org/bits

KeyboardInterrupt: 

In [22]:
coll_knowledge_notes = reports_coll.find({'collection':'Knowledge Notes'},no_cursor_timeout=True)
reports_coll.count_documents({})

8266

In [23]:
8266//99

83

In [ ]:
coll_knowledge_notes = reports_coll.find({'collection':'Knowledge Notes'},no_cursor_timeout=True)

for element in coll_knowledge_notes[start_index:start_index+100]:
    print(start_index)
    
    url = element["pdf_link"]
    object_id = element['_id']
    storage_dir = 'tempstorage'
    title = element["title"]
    start_index+=1
    
    print(url)
    print(title)
    if url:
        if 'full_text' not in element.keys():
            try:
                pdf_path = download_pdf(url,storage_dir,title)
                pdf_to_images(pdf_path,storage_dir,object_id,reports_coll)
                master_text = images_to_text(storage_dir)
                upload_text_to_mongodb(reports_coll,object_id,master_text)
                delete_temp_content(storage_dir)
                time.sleep(60)
            except:
                print('ERROR - trying again')
                time.sleep(300)
                try:
                    pdf_path = download_pdf(url,storage_dir,title)
                    pdf_to_images(pdf_path,storage_dir,object_id,reports_coll)
                    master_text = images_to_text(storage_dir)
                    upload_text_to_mongodb(reports_coll,object_id,master_text)
                    delete_temp_content(storage_dir)
                except:
                    print('ERROR AGAIN WITH THIS DOCUMENT - MOVING ON')
                    pass


In [100]:
start_index

210

In [75]:
coll_knowledge_notes = reports_coll.find({'collection':'Knowledge Notes'})
n = 0
for i in coll_knowledge_notes[1:2]:
    print(i)



{'_id': ObjectId('5f770a0a9d1d71848dfe8814'), 'title': 'COVID-19 Crisis Through a Migration Lens', 'abstract': 'The economic crisis induced by COVID‐19 could be long, deep, and pervasive when viewed through amigration lens. Lockdowns, travel bans, and social distancing have brought global economic activities to a near standstill. Host countries face additional challenges in many sectors, such as health and agriculture, that depend on the availability of migrant workers. Migrants face the risk of contagion and also the possible loss of employment, wages, and health insurance coverage. This Migration and Development Brief provides a prognosis of how these events might affect global trends in international economic migration and remittances in 2020 and 2021. Considering that migrants tend to be concentrated in urban economic centers (cities), and are vulnerable to infection by the coronavirus, there is a need to include migrants in efforts to fight thecoronavirus. Migrant remittances prov